In [ ]:
import numpy as np
import pandas as pd
import multiprocessing

import scipy
from scipy import spatial
from scipy.spatial import cKDTree

import sklearn as sk
from sklearn import svm
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn import clone
from sklearn.externals.six.moves import xrange

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline

%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

import dask.dataframe as ddf
import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

In [ ]:
# 2018: SS18/FW18 ranges

# SS18 = pd.read_excel('RangeSS18.xlsx', skiprows = 5) # **PROBLEM**
SS18 = pd.read_excel('SS18_range_data.xlsx') # Solution: Artem file
FW18 = pd.read_excel('RangeFW18.xlsx', skiprows = 5)

SS18_range = SS18['Article Number'].unique()
FW18_range = FW18['Article Number'].unique()

In [ ]:
# SS20 = (pd.read_excel('SS20Range.xlsx', skiprows = 6).
#        drop(['Unnamed: 0', 'vlookup'], axis = 1))

# SS20_all_articles = SS20['Article Number'].unique()

# SS20 = SS20[SS20['CO'] == 'Y']
# dat['CO'].fillna('N', inplace = True)
# dat['SS19 Ranged'].fillna('N', inplace = True)
# dat['FW19 Ranged'].fillna('N', inplace = True)

# dat.to_csv('dat_SS20_range.csv')

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique() # 1340

In [ ]:
# EDA
dat_SS20_range = dat_SS20_range[['Article Number', 'Model', 'Product Division', 
                                 'Category Marketing Line', 'Gender', 'Model Number', 'Product Group',
                                 'Product Type', 'Sports Category', 'Key Category', 'WE eCom', 'SS19 Ranged', 
                                 'FW19 Ranged', 'CO']]


In [ ]:
dat_SS20_range.groupby(['SS19 Ranged', 'FW19 Ranged', 'CO'])['Article Number'].count()
# dat_SS20_range['Key Category'].value_counts()[:10]
# dat_SS20_range['Product Division'].value_counts()
# dat_SS20_range['Product Group'].value_counts()[:10]
# dat_SS20_range['Gender'].value_counts()

# dat_SS20_range[dat_SS20_range['FW19 Ranged'] == 'Y'].groupby('Article Number')[]

## Corrections

In [111]:
%%time

dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

CPU times: user 2min 4s, sys: 6.02 s, total: 2min 10s
Wall time: 2min 7s


In [ ]:
dat = dat0.copy()

In [ ]:
# EDA: clearance transactions by season
dat.groupby('season')['Clearance'].value_counts()

In [ ]:
# Transaction subset: SS19
dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]
dat = dat[dat['season'] == 'SS19']

In [ ]:
# Wall time: 38.7 s

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique()
dat = dat[[(a in SS20_range) for a in dat['article_number']]]

In [ ]:
clearance = dat.copy() # FOR CLEARANCE CORRECTION IN NEXT SECTION

# 'aggregate' to weekly sums by article for buy_availability merge and adjustment
dat.set_index('consumer_order_date', inplace = True)

dat = dat[['article_number', 'gross_demand_quantity']].groupby(['article_number']).resample('W').sum()
dat.reset_index(inplace=True)

# Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

In [ ]:
# Wall time: ~40 s

# 2018 data --> SS19 in Dec. 2018 -- to merge with 2019 SS19
dat_stock = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])
dat_stock = dat_stock[dat_stock['min_date_of_week'] > pd.to_datetime('2018-12-1')] # filter to > Dec1 (SS19)

# 2019 data
dat_stock2 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0) # *** DATA ***

# Build: Dec2018 + 2019
dat_stock = pd.concat([dat_stock, dat_stock2])

# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]

dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [ ]:
# weekly demand + buy_availability -- for understock correction
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'outer')

# dat['buy_availability'] = dat['buy_availability'].fillna(1)

In [ ]:
# ----- Calculate *observed* full season gross_qty per article -----
dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_demand_quantity'].sum())

dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [ ]:
# WEEKLY averages for articles when fully stocked
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_demand_quantity'].
    mean())

dat_stocked.rename(columns = {'gross_demand_quantity':'stocked_weekly_avg_gross_demand_quantity'}, inplace= True)

dat_stocked['understock_correction'] = 26*dat_stocked['stocked_weekly_avg_gross_demand_quantity']

In [ ]:
dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.drop('stocked_weekly_avg_gross_demand_quantity', inplace=True, axis = 1)

In [ ]:
# ------ OVERSTOCK CORRECTION ------

In [ ]:
clearance0 = clearance[clearance['Clearance'] == 0]

clearance0 = pd.DataFrame(clearance0.groupby(['article_number'])['gross_demand_quantity'].sum())

clearance0.rename(columns = {'gross_demand_quantity': 'overstock_correction'}, inplace = True)

In [ ]:
# EDA -- article intersection -- articles in (i) SS20 range AND (ii) in SS19 clearance transactions
set(SS20_range).intersection(set(clearance[clearance['Clearance'] == 1].reset_index()['article_number'])) # 37

In [ ]:
dat_season = pd.merge(dat_season, clearance0, how = 'outer', left_index = True, right_index = True)

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0)) # *** DATA ***
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

preds = preds[preds['season'] == 'SS19']

dat_season = pd.merge(
    dat_season, preds, 
    left_on = ['article_number'],
    right_on=['article'], 
    how = 'left').round()

dat_season = dat_season[['article', 'season', 'season_gross_demand_quantity', 'understock_correction',
                        'overstock_correction', 'ecom_marketing_forecast']]

In [ ]:
def projection(gross, forecast, under, over):
    if forecast < gross:
        return under
    elif forecast >= gross:
        return 1.2*over
    else:
        return gross

dat_season.loc[:,'projected_gross_demand_quantity'] = (
    dat_season.apply(lambda row: projection(row['season_gross_demand_quantity'], 
                                            row['ecom_marketing_forecast'], 
                                            row['understock_correction'],
                                            row['overstock_correction']), 
                     axis = 1))

In [ ]:
# Article subset: projected to have gross_demand_quantity > 100 in SS19
dat_season = dat_season[dat_season['season_gross_demand_quantity'] > 83]

In [ ]:
dat_season['SS20_prediction'] = dat_season['projected_gross_demand_quantity']*1.1

In [ ]:
dat_season.shape
dat_season

In [ ]:
dat_season.to_csv('article_baselines.csv')

In [ ]:
print('Projected Gross Demand Quantity summary')
dat_season['projected_gross_demand_quantity'].describe()
print()
print('Summary: eCom forecast minus projected gross demand')
(dat_season['ecom_marketing_forecast'] - dat_season['projected_gross_demand_quantity']).describe()


In [ ]:
# Note: get for Artem over/understock correction for 'B41674', CE2441


## Overstock-SupplyChain-Understock (OSU) Correction

In [ ]:
%%time

dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

In [ ]:
dat = dat0.copy()

In [ ]:
# Transaction subset: SS19
dat = dat[['consumer_order_date', 'article_number', 'gross_demand_quantity', 'Clearance', 'season']]
dat = dat[dat['season'] == 'SS19']

In [ ]:
# Wall time: 38.7 s

dat_SS20_range = pd.read_csv('dat_SS20_range.csv') # *** DATA ***
SS20_range = dat_SS20_range['Article Number'].unique()
dat = dat[[(a in SS20_range) for a in dat['article_number']]]

In [ ]:
# Subset to non-clearance transactions only --- change here for David/Mike's request
dat = dat[dat['Clearance'] == 0]

In [ ]:
# 'aggregate' to weekly sums by article for buy_availability merge and adjustment
dat.set_index('consumer_order_date', inplace = True)

dat = dat[['article_number', 'gross_demand_quantity']].groupby(['article_number']).resample('W').sum()
dat.reset_index(inplace=True)

# Add 'week' and 'year' for merging with stock (buy_availability) data (b/c min_date_of_week)
dat['week'] = [t.week for t in dat['consumer_order_date']]
dat['year'] = [t.year for t in dat['consumer_order_date']]

In [ ]:
# Wall time: ~40 s

# 2018 data --> SS19 in Dec. 2018 -- to merge with 2019 SS19
dat_stock = pd.read_csv('Stock Data/Stock data 2018.csv', low_memory=False, index_col = 0) # *** DATA ***
dat_stock['min_date_of_week'] = pd.to_datetime(dat_stock['min_date_of_week'])
dat_stock = dat_stock[dat_stock['min_date_of_week'] > pd.to_datetime('2018-12-1')] # filter to > Dec1 (SS19)

# 2019 data
dat_stock2 = pd.read_csv('Stock Data/Stock data 2019.csv', low_memory=False, index_col = 0) # *** DATA ***

# Build: Dec2018 + 2019
dat_stock = pd.concat([dat_stock, dat_stock2])

# Tidy
dat_stock.reset_index(inplace = True)
dat_stock.drop(['avg(ecom_available_stock)', 'avg(size_availability)'], axis = 1, inplace = True)
dat_stock.rename(columns = {'min_date_of_week': 'date', 'avg(buy_availability)': 'buy_availability'}, inplace = True)

# Filter to SS20_range carryovers
dat_stock = dat_stock[[(a in SS20_range) for a in dat_stock['article_number']]]

dat_stock['date'] = pd.to_datetime(dat_stock['date'])

# For merging with transaction data
dat_stock['week'] = [t.week for t in dat_stock['date']]
dat_stock['year'] = [t.year for t in dat_stock['date']]

In [ ]:
# Merge weekly demand df and buy_availability df -- for understock correction
dat = pd.merge(dat, dat_stock, 
               left_on = ['article_number', 'year', 'week'], 
               right_on = ['article_number', 'year', 'week'],
               how = 'outer')

In [ ]:
# ----- Calculate *observed* full season gross_demand_quantity per article -----

dat_season = pd.DataFrame(dat.groupby(['article_number'])['gross_demand_quantity'].sum())
dat_season.rename(columns = {'gross_demand_quantity':'season_gross_demand_quantity'}, inplace = True)

In [ ]:
%whos DataFrame ndarray

In [ ]:
'DU0367' in SS20_range

In [ ]:
dat_stock[dat_stock['article_number'] == 'DU0367']
dat_season.loc['DU0367']
dat[dat['article_number'] == 'DU0367']

In [ ]:
# WEEKLY averages for articles when buy_availability > 0.35
dat_stocked = pd.DataFrame(
    dat[dat['buy_availability'] > 0.35].
    groupby(['article_number'])['gross_demand_quantity'].
    mean())

dat_stocked.rename(columns = {'gross_demand_quantity':'corrected_weekly_avg_gross_demand_quantity'}, inplace= True)

# Extend to full season (26 weeks) to estimate full season demand
dat_stocked['corrected_gross_demand_quantity'] = 26*dat_stocked['corrected_weekly_avg_gross_demand_quantity'] 

In [ ]:
dat_season = pd.merge(dat_season, dat_stocked, left_index=True, right_index=True, how = 'outer')
dat_season.drop('corrected_weekly_avg_gross_demand_quantity', inplace=True, axis = 1)

In [ ]:
preds = (pd.DataFrame(
    pd.read_csv('Buyers predictions.csv', low_memory=False, index_col = 0)) # *** DATA ***
         [['season', 'ecom_marketing_forecast']].
         reset_index().
        dropna())

preds = preds[preds['season'] == 'SS19']

dat_season = pd.merge(
    dat_season, preds, 
    left_on = ['article_number'],
    right_on=['article'], 
    how = 'left').round()

dat_season = dat_season[['article', 'season', 'season_gross_demand_quantity', 
                         'corrected_gross_demand_quantity', 'ecom_marketing_forecast']]

In [ ]:
dat_season = dat_season[dat_season['season_gross_demand_quantity'] > 83]

dat_season['DAA_SS20_prediction'] = dat_season['corrected_gross_demand_quantity']*1.1

In [ ]:
dat_season.shape
dat_season

In [ ]:
# dat_season.to_csv('dat_season.csv')

### SS20 buy numbers

In [ ]:
# Wall time: 55.8 s

# SS20 buyer predictions
# SS20_buys = pd.read_excel('SS20 RMA2_May7.xlsx', skiprows = 2, sheet_name= '03 May download')
# SS20_buys = SS20_buys[['Article Number', 'Model', 'Market', 'WE eCom', 
#                        'Sub-Brand', 'Product Division', 'Marketing Segment']]


# SS20_buys.rename(columns = {'WE eCom': 'eCom SS20 Forecast'}, inplace = True)
# SS20_buys = SS20_buys[[(a in AOIs) for a in SS20_buys['Article Number']]]
# SS20_buys.to_csv('SS20_buys_carryovers.csv')

In [ ]:
dat_season = pd.read_csv('dat_season.csv', low_memory=False, index_col = 0)
SS20_buys = pd.read_csv('SS20_buys_carryovers.csv', low_memory=False, index_col = 0)

In [ ]:
dat_season = (pd.merge(dat_season, SS20_buys[['Article Number', 'eCom SS20 Forecast']], 
                       left_on='article', right_on='Article Number', how = 'outer'))

dat_season = dat_season[['article', 'Article Number', 'season_gross_demand_quantity', 
                         'corrected_gross_demand_quantity', 'ecom_marketing_forecast',
                        'DAA_SS20_prediction', 'eCom SS20 Forecast']]

dat_season['prediction_disparity'] = dat_season['eCom SS20 Forecast'] - dat_season['DAA_SS20_prediction']
dat_season['abs_prediction_disparity'] = abs(dat_season['eCom SS20 Forecast'] - dat_season['DAA_SS20_prediction'])

In [105]:
# Add cost to DF
cost_price = pd.read_csv('Cost Price.csv', low_memory=False, index_col = 0)

cost_price = cost_price[[(a in SS20_range) for a in cost_price.index]]
cost_price.rename(columns = {'avg(cost_of_sales)': 'cost'}, inplace = True)
cost_price = pd.DataFrame(cost_price['cost'].groupby(cost_price.index).mean()).round()
dat_season = pd.merge(dat_season, cost_price, left_on = 'article', right_index = True)

In [ ]:
# Add price to DF
dat0 = pd.read_csv('Transaction data.csv', low_memory=False) # *** DATA ***
dat0['consumer_order_date'] = pd.to_datetime(dat0['consumer_order_date'])

In [112]:
dat0.head()

,consumer_order_date,country,article_number,brand,gross_demand_quantity,net_qty,gross_sales_gross_disc_net_ret,net_sales,total_markdown,article_promotion_main_category_group,fraction_of_full_price,markdown,Clearance,FW_or_SS,season
0,2016-09-02 22:00:00,ES,AB0659,REEBOK,1.0,1.0,8.26,15.18,6.92,Outlet,0.544137,1,1,FW,FW16
1,2016-11-20 23:00:00,ES,AB0661,REEBOK,1.0,1.0,8.26,14.17,5.91,Outlet,0.582922,1,1,FW,FW16
2,2017-03-31 22:00:00,ES,AJ8191,REEBOK,2.0,2.0,29.66,34.13,4.47,Outlet,0.869030,1,1,SS,SS17
3,2017-03-31 22:00:00,ES,AJ8194,REEBOK,1.0,1.0,14.83,15.17,0.34,Outlet,0.977587,1,1,SS,SS17
4,2017-04-13 22:00:00,ES,AJ8017,REEBOK,1.0,1.0,14.83,18.96,4.13,Outlet,0.782173,1,1,SS,SS17


In [ ]:
# Add 'avg_price' to table
# (1) Filter to the 491 SS20 articles of interest
# (2) sale price on THAT transaction: gross_sales_gross_disc_net_ret/net_qty
# (3) find average of that per article
# (4) add that to table

In [ ]:
# Problems:
# (1) DU0367 has 'corrected = NaN'

In [ ]:
# Way under: everything
# Way over: DB3258, DP2398, EF2803, DQ3319